$$\newcommand{\trans}[1]{g#1}$$

# Part 2: IIC loss function

## About mutual information

Now let's discuss what is __mutual information__ and how can it be estimated.

A __mutual information__ is a functional which takes as input a joint probability distribution of two random variables. That means, the for a pair of random variables it takes as input the probabilities for all pairs of values and outputs a number. Let's

Before we start digging into it let's recall the notion of __joint probabilities__. Denote the probability of random variables $A$ and $B$ to take values $a$ and $b$ respectfully as $p_{AB}(a,b)$. Than the set of values $p_{AB}(a,b)$ for all possible $a$ and $b$ is called a __joint probability  distribution__. 

For our case it's sufficient to assume, that the random variables are discrete. Let's also assume, that the variables $A$ and $B$ can take the values 0, 1 .. $N_C$. 

 
The next notion to remember is the __marginal probability distributions__ . For random variable x it is defined as

$$ p_{A}(a) = \sum_{b=0}^{N_C} p_{AB}(a,b)$$

The marginal probability distribution of $A$ tells us, how often $A$ is going to take this or that value, if we are not looking at $B$ at all.
For $B$ __marginal probability__ distribution is defined in a similar manner:
$$ p_{B}(b) = \sum_{a=0}^{N_C} p_{AB}(a,b)$$

Now we are ready to define a mutual information. It is given by a formula:

$$ I(A,B) = \sum_{a=0}^{N_C} \sum_{b = 0}^{N_C} p_{AB}(a, b) \log \frac{p_{AB}(a, b)}{ p_A(a) p_B(a)} $$

The formula may seem complicated, but the motivation is simple. To see it let's consider several properties of mutual information:

1) Mutual information us symmetric:  $I(A, B) = I(B,A)$

1) Mutual information is zero if and only if $A$ and $B$ are independent.  This makes perfect sense, since independent variables should not carry any information about each other.

2) Mutual information is also not less than zero. Together with the previous property it implies, that mutual information of dependent variables is often larger, than of independent ones. 

4) For a given random variable $A$ mutual information is maximized when $A$ is completely defined by  $B$ in a non-random manner. Consequently, the variables, that completely define each other maximize mutual information. 



<details><summary>Explanation of the third property </summary>
<p>

The mutual information can be expressed with the help of [entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) and [conditional entropy](https://en.wikipedia.org/wiki/Conditional_entropy) as:
    
$$ I(A,B)  = H(A) - H(A|B) $$

If $A$ is a  random variable with a given distribution, $H(A)$ is constant. Conditional entropy $ H(A|B) $ is always non-negative. Moreover, if $A$ is defined by $B$ (which means that after measuring $B$ we know exactly which value takes $A$), than $ H(A|B) = 0$, thus in this case $I(A,B)$ takes its maximum value equal to $H(B)$.
</p>
</details>

Now it's clear why mutual information is a good measure of dependence of two random variables. The more interconnected they are, the bigger the quantity and vice versa.

## Mutual information estimation

As you can see from above, to calculate mutual information it's sufficient to estimate a joint probability function $ p_{A,B}(a,b) $. After this it's straightforward to calculate the marginal distributions and after that the mutual information itself. Before showing you this estimation I want to make a disclaimer: the formulae below should  be taken not as an absolute truth, but rather a model assumption.  

Let's denote $i$-th data sample as $x_i$, the corresponding transformed data sample as $ gx_i$ and the encoder function as $\Phi(\cdot)$. After the encoder processes the sample it gives a set of $N_C$ numbers ($N_C$ is equal to 10 for MNIST). We denote the entire set as $ \Phi(x_i) $ for original data and $ \Phi(gx_i) $ for  transformed data. :




$$\Phi(x_i) = \begin{pmatrix} \Phi_0(x_i) \\ \Phi_1(x_i) \\ \vdots \\ \Phi_{N_C}(x_i) \end{pmatrix} ~~~~~~~\Phi(gx_i) = \begin{pmatrix} \Phi_0(gx_i) \\ \Phi_1(gx_i) \\ \vdots \\ \Phi_{N_C}(gx_i) \end{pmatrix}$$
  
The parameter $N_C$ here has a clear meaning -- it's a number of cluster/classes in out problem.

We treat these vectors as probability distributions, conditioned on the $i$-th sample:

$$ P_{orig}(a|i) = \Phi_a(x_i) ~~~~~~ P_{trans}(a|i) = \Phi_a(gx_i) $$

To get the unconditional joint probabilities, we need to multiply the probabilities and to average over the data:

$$ P_{orig, trans}(a,b) = \frac{1}{N_B} \sum_{i \in batch}  \Phi_a(x_i) \Phi_b(\trans{x}_i) $$

where $N_B$ is a batch size. 

<details><summary>For the cautious</summary>
 The multiplication step is not clear as it implies that the conditional probabilities $\Phi(x_i)$ and  $\Phi(\tilde{x}_i)$ correspond to the independent random variables, which is not intuitively evident. To defend it one may say, that even if the conditioned random variables are independent, they can lead to correct distributions when only the "correct" class has high probability.  Perhaps we should treat that /step as a educated guess and not to insist on a complete mathematical rigor here. If you know, how to justify it or how to treat it in a better way, contact me please.
    
</details>
        

In IIC setting we expect the joint probability to be a symmetric function: the probability to observe the pair of classes $(a,b)$ should be the same s the probability to observe the pair $(b,a)$. However in general case the above formula for the joint probabilities doesn't have this symmetry.  To fix the situation we perform a symmetrization:
$$P^{sym}_{orig, trans}(a,b) = \frac{1}{2}\left(P_{orig, trans}(a,b) + P_{orig, trans}(b,a) \right). $$
    
We will omit the upper index "$sym$" further on.


Now the "vague model assumptions" part is over. After obtaining joint probability distribution it's straightforward to calculate the marginals:

$$  P_{orig} (a) = \sum_{b=0}^{N_C}P_{orig, trans}(a,b) ~~~~~~~ P_{trans} (b) = \sum_{a=0}^{N_C}P_{orig, trans}(a,b) $$

And the mutual information:

$$ I (orig, trans) = \sum_{a=0}^{N_C} \sum_{b = 0}^{N_C} P_{orig, trans}(a,b) \log \frac{P_{orig, trans}(a,b)} {P_{orig} (a) P_{trans} (b)}$$ 

Our encoder $\Phi(\cdot)$  has a huge number of parameters, adjusting which we can try to make this function as large as we can. This expression is a differentiable function and thus can be optimized with any gradient method you prefer.

## Entropy vs degeneracy

Sometimes in IIC setting the model tend to group the data to a number of clusters much less than the desired. The worst case of this scenario is when all the samples get into one cluster. This situation is called clustering degeneracy and appears not only in IIC, but in other clustering methods. 

Sometimes it possible to fight it by slightly modifying the Mutual information formula. To see, how can it be done, let's recall a concept of __entorpy__. The [entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) $H(A)$ of random variable $A$ is the defined as:

$$H(A) = -\sum_{a=0}^{N_C} p_A(a) \log(p_A(a))$$

The entropy  can be seen as a measure of disorder of a random variable. It is maximized when all the samples are distributed equally within the clusters/classes. 

The [conditional entropy](https://en.wikipedia.org/wiki/Conditional_entropy) $H(A|B)$ is defined as:

$$H(A|B) = -\sum_{a=0}^{N_C} \sum_{b=0}^{N_C} p_{AB}(a,b) \log\frac{p_{AB}(a,b)}{p_A(a)}$$

It is minimized when the measurement of $B$ completely predicts the measurement of $A$.

Using the entropies the mutual information can be expressed as:

$$ I(A,B) = \frac{1}{2}\big(H(A) + H(B)\big) - \frac{1}{2}\big(H(A|B) + H(B|A)\big)$$

The optimization of mutual information can be seen as a fight between marginal (usual) if conditional entropies. If all samples tend to group more intensely than required, we may want to  help the first term by adding the additional weight to it. So the modification of the mutual information should be:

$$I_\lambda(A,B) = I(A,B) +(\lambda - 1 )\big(H(A) + H(B)\big)$$

When the $\lambda$ is equal to 1, the modified mutual information is equal to the original mutual information. Making $\lambda$ greater than one pushes the model towards spreading the labels to different clusters. 

## Conclusion

Wuf! We have finished to study mutual inforamtion loss. It's time to switch to practice -- to the [Part 3](link_to_the_third_part).